<a href="https://colab.research.google.com/github/tenzindiki/CLPS-950-Group-Project/blob/main/Python_Typing_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import numpy as np
import tkinter as tk
from tkinter import ttk
import pygame
import json
import requests
import math
import threading
import os
import random
import time
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio, display

pygame 2.6.1 (SDL 2.28.4, Python 3.11.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Setting Up Test (Initialization)

In [ ]:
start_time = None # tracks elapsed time (starts at 0)
char_position = 0 # tracks position of character within a string of text (starts at 0)
errors = 0 # tracks number of mistakes made by user (starts at 0)
wpm_tracker = [] # tracks words per minute values as they change throughout a given test
character_mistype = defaultdict(int) # tracks number of times user incorrectly types a particular character
word_counter = Counter() # tracks number of times a particular word emerges during the test
total_characters = 0 # tracks total number of characters encountered during a test (starts at 0)


# Loading Text

In [ ]:
API_URL = "https://quoteapi.pythonanywhere.com/" # test retrieves quotations from this site
def retrieve_quotation(): # function retrieves quotation
  try:
    quotation = requests.get(API_URL) # requests quotation from our chosen API
    if quotation.ok: # if quotation can be retrieved
      text = quotation.json()['content'] # specifically transfers quotation from API to Python; JSON string is converted into Python object
      for word in text.split(): # isolates each word in the text
        word_counter[word.lower()] += 1 # every time a particular word appears, a word counter is updated to keep track of how many times that word has appeared during the test; also makes sure each word is lowercase
      return text
  except: # if quotation cannot be retrieved
    fallback = random.choice(["We hope you're enjoying our typing test!","Feel free to share this test with your friends to see whose typing reigns supreme!"]) # example sentences that user can type if quotations do not load
    for word in fallback.split(): # isolates each word in the text
      word_counter[word.lower()] += 1 # every time a particular word appears, a word counter is updated to keep track of how many times that word has appeared during the test; also makes sure each word is lowercase
    return fallback

# Establishing Proximity of Letters (Relative to Each Other)

In [ ]:
keyboard_proximity = { # two letters were considered close to each other -- i.e., in near proximity -- when they were adjacent to each other, whether horizontally, vertically, or diagonally
    'a':['q','w','s','z'],'s':['a','w','e','d','x','z'],'d':['s','e','r','f','c','x'],'f':['d','r','t','g','v','c'],'g':['f','t','y','h','b','v'],'h':['g','y','u','j','n','b'],'j':['h','u','i','k','m','n'],'k':['j','i','o','l','m'],'l':['k','o','p'],'q':['w','a'],'w':['q','e','s','a'],'e':['w','r','d','s'],'r':['e','t','f','d'],'t':['r','y','g','f'],'y':['t','u','h','g'],'u':['y','i','j','h'],'i':['u','o','k','j'],'o':['i','p','l','k'],'p':['o','l'],'z':['a','s','x'],'x':['z','s','d','c'],'c':['x','d','f','v'],'v':['c','f','g','b'],'b':['v','g','h','n'],'n':['b','h','j','m'],'m':['n','j','k']
}

# Constructing GUI

In [ ]:
class PythonTypingTestApp: # defines class of GUI
  def __init__(self,root): # initializes new objects within the class
    self.root = root # sets up and saves main window of typing test
    self.root.title("Python Typing Test") # creates title of typing test
    self.root.geometry("900x500") # specifies size of window

    self.setup_widgets() # places the elements of our GUI within the test (detailed in next section of the code)
    self.reset_test() # clears all previous data before (re)starting test

    #just testing to see that this is updating and working properly